The purpose of GST's fiducials is to generate an informationally-complete set of states and measurements. However, it's possible to _reduce_ the number of fiducials necessary to do so.

In this tutorial, we use the `pygsti.alg.find_sufficient_fiducial_pairs` function to reduce the number of fiducials necessary for GST.

In [1]:
import pygsti
import json

In [2]:
# Follow Algorithm tutorial to generate LGST gatesets
gs_target = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")
ds = pygsti.io.load_dataset("tutorial_files/Example_Dataset.txt", cache=True)
fiducialList = pygsti.io.load_gatestring_list("tutorial_files/Example_FiducialList.txt")

#Run LGST to get an initial estimate for the gates in gs_target based on the data in ds
specs = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
gs_lgst = pygsti.do_lgst(ds, specs, targetGateset=gs_target, svdTruncateTo=4, verbosity=1)

#Gauge optimize the result to match the target gateset
gs_lgst_after_gauge_opt = pygsti.optimize_gauge(gs_lgst, "target", targetGateset=gs_target)

#Contract the result to CPTP
gs_clgst = pygsti.contract(gs_lgst_after_gauge_opt, "CPTP")

#Get lists of gate strings for successive iterations of LGST to use
specs  = pygsti.construction.build_spam_specs(fiducialGateStrings=fiducialList)
germList = pygsti.io.load_gatestring_list("tutorial_files/Example_GermsList.txt")
maxLengthList = json.load(open("tutorial_files/Example_maxLengths.json","r"))

Loading from cache file:  tutorial_files/Example_Dataset.txt.cache


In [3]:
#Get sufficient set of fiducial pairs, meaning that with these fidicual pairs and the given set of germs,
# the number of gateset parameters which are amplified when all pairs are used are also amplified when using
# the returned subset.
fidPairs = pygsti.alg.find_sufficient_fiducial_pairs(gs_target, fiducialList, fiducialList, germList, verbosity=1)
print fidPairs

------  Fiducial Pair Reduction --------
maximum number of amplified parameters = 

 34
Beginning search for a good set of 1 pairs (36 pair lists to test)


Beginning search for a good set of 2 pairs (630 pair lists to test)


Beginning search for a good set of 3 pairs (7140 pair lists to test)


[(0, 0), (0, 1), (1, 0)]


In [4]:
#Test a specific set of fiducial pairs: see how many gateset parameter are amplified
#FPR.find_sufficient_fiducial_pairs(gs_target, fiducialList, germList, testPairList=[(0,0),(0,1),(1,0)], verbosity=4)

In [5]:
lsgstListOfLists = pygsti.construction.make_lsgst_lists(gs_target.gates.keys(), fiducialList, fiducialList,
                                                        germList, maxLengthList, fidPairs)

gs_lsgst_list = pygsti.do_iterative_mc2gst(ds, gs_clgst, lsgstListOfLists, verbosity=2,
                                         minProbClipForWeighting=1e-6, probClipInterval=(-1e6,1e6),
                                         returnAll=True )


--- Iterative MC2GST: Beginning iter 1 of 10 : 92 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 48.2157 (92 data params - 40 model params = expected mean of 52; p-value = 0.623488)



--- Iterative MC2GST: Beginning iter 2 of 10 : 92 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 48.2156 (92 data params - 40 model params = expected mean of 52; p-value = 0.623489)



--- Iterative MC2GST: Beginning iter 3 of 10 : 95 gate strings ---


...

--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 233.121 (242 data params - 40 model params = expected mean of 202; p-value = 0.0657696)



--- Iterative MC2GST: Beginning iter 9 of 10 : 274 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 267.547 (274 data params - 40 model params = expected mean of 234; p-value = 0.065175)



--- Iterative MC2GST: Beginning iter 10 of 10 : 306 gate strings ---


--- Minimum Chi^2 GST ---
  Sum of Chi^2 = 291.434 (306 data params - 40 model params = expected mean of 266; p-value = 0.136086)

In [6]:
# Compute a few additional quantities needed to generate the report
Ls = maxLengthList
gateStrDict = { (L,germ):pygsti.construction.repeat_with_max_length(germ,L,False) for L in Ls for germ in germList }

#remove duplicates by replacing duplicate strings with None
runningList = []
for L in Ls:
    for germ in germList:
        if gateStrDict[(L,germ)] in runningList:
            gateStrDict[(L,germ)] = None
        else: runningList.append( gateStrDict[(L,germ)] )

In [7]:
res = pygsti.report.Results()
res.init_Ls_and_germs("chi2", gs_target, ds, gs_clgst, maxLengthList, germList,
                    gs_lsgst_list, lsgstListOfLists, fiducialList, fiducialList, 
                    pygsti.construction.repeat_with_max_length, False, fidPairs)

res.create_full_report_pdf(filename="tutorial_files/Example_report_FR.pdf", verbosity=2)

*** Generating tables ***


Generating table: targetSpamTable


Generating table: targetGatesTable


Generating table: datasetOverviewTable


Generating table: bestGatesetSpamTable


...

2 

 Generating figure: invertedBestEstimateColorBoxPlot


*** Merging into template file ***


Latex file(s) successfully generated.  Attempting to compile with pdflatex...


Initial output PDF tutorial_files/Example_report_FR.pdf successfully generated.
Final output PDF tutorial_files/Example_report_FR.pdf successfully generated. Cleaning up .aux and .log files.